#**Similarity Search using text and image embedding**
**What is CLIP?**

CLIP (Contrastive Language–Image Pretraining) is a multimodal model developed by OpenAI. It takes an image and a text input, encodes both into a shared embedding space, and measures how well they match. It’s trained to associate textual descriptions with visual concepts.

In [1]:
! pip install transformers torch pillow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 106.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 86.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 63.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 70.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

In [3]:
from PIL import Image
from transformers import CLIPProcessor, CLIPModel
import torch
import requests

##**1. Load the Model and Processor**

In [4]:
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/605M [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


...

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/862k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

##**2. Download and Load the Image**

In [42]:
#image_url = "https://upload.wikimedia.org/wikipedia/commons/9/99/Sample_User_Icon.png"
image_url = "https://upload.wikimedia.org/wikipedia/commons/1/18/Dog_Breeds.jpg"
image = Image.open(requests.get(image_url, stream=True).raw)

##**3. Prepare Text Descriptions**

In [28]:
texts = [
    "a user avatar",
    "a person icon",
    "a dog",
    "a landscape image",
    "an icon of a person"
]


##**4. Preprocess the Image and Texts Together**

In [29]:
inputs = processor(text=texts, images=image, return_tensors="pt", padding=True)

##**5. Compute Embeddings**

In [30]:
with torch.no_grad():
    outputs = model(**inputs)
    image_embeds = outputs.image_embeds
    text_embeds = outputs.text_embeds

In [31]:
print("Image:",image_embeds)
print("Text:",text_embeds)

Image: tensor([[-9.5081e-03, -9.0552e-03,  1.7284e-02, -2.8210e-02,  3.4596e-02,
         -7.6915e-02,  5.7895e-02,  7.4794e-03, -5.4263e-03,  1.2266e-02,
         -2.4470e-02,  6.0751e-03,  2.9895e-02, -1.2416e-02,  6.2213e-02,
         -9.1396e-04,  6.9065e-02, -7.7196e-03,  2.7649e-02,  1.0681e-02,
         -4.7986e-02, -1.4231e-02,  2.4457e-02, -4.5163e-02, -2.4123e-02,
         -8.4966e-03,  1.2040e-02, -8.7617e-04, -1.3559e-02,  1.3047e-03,
          1.1148e-02,  3.3815e-02, -1.8869e-02, -2.3130e-02,  6.6858e-03,
          3.9781e-02, -1.0485e-05,  4.2415e-02, -4.1874e-02,  1.2993e-01,
         -6.2737e-02, -3.4650e-02, -2.4691e-02, -1.7027e-02, -1.6323e-03,
         -1.2661e-02, -1.2451e-02, -1.6207e-03,  2.9761e-02, -1.0381e-02,
          2.9939e-03,  3.7208e-02,  7.1424e-03, -1.9409e-02,  1.3242e-02,
          1.5935e-02,  2.0652e-02,  3.3049e-02, -3.4423e-02, -1.6699e-02,
          1.2412e-01,  1.6245e-02,  4.8168e-02,  2.0641e-02, -1.7104e-02,
         -4.2634e-02, -7.2182e-

##**6. Compute Similarity Scores**

In [32]:
similarities = torch.nn.functional.cosine_similarity(image_embeds, text_embeds)

##**7. Display Results**

In [33]:
for text, score in zip(texts, similarities):
    print(f"'{text}' similarity: {score.item():.4f}")

'a user avatar' similarity: 0.2144
'a person icon' similarity: 0.2213
'a dog' similarity: 0.2643
'a landscape image' similarity: 0.1970
'an icon of a person' similarity: 0.2102
